In [1]:
import numpy as np
import os

import random
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import astropy.units as u
import glob
from ast import literal_eval
import astropy.constants as const

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *
from plot_binned_detection_1d2d_0526 import *

In [3]:

detectors = ['SURF', 'CJPL', 'Kamioka', 'LNGS',  'SNOlab']
metallicities = ['high', 'low']
power = 0.9
alphas = [0.001, 0.01, 0.1 ]
nuclei = 'Xenon'
file_name =  'atmhepdsnb_Xe_DT_optimized_consecutive_thrd0_numeNa_S1S2binning.csv'
DT_pd = pd.read_csv(file_name, index_col = 0 )
DT_pd

,atm_CJPL_high,atm_CJPL_low,atm_Kamioka_high,atm_Kamioka_low,atm_LNGS_high,atm_LNGS_low,atm_SURF_high,atm_SURF_low,atm_SNOlab_high,atm_SNOlab_low,...,dsnb_CJPL_high,dsnb_CJPL_low,dsnb_Kamioka_high,dsnb_Kamioka_low,dsnb_LNGS_high,dsnb_LNGS_low,dsnb_SURF_high,dsnb_SURF_low,dsnb_SNOlab_high,dsnb_SNOlab_low
"(0.001, 0.9)",3503.499280,3506.303876,2362.853878,2470.731870,1514.438044,1513.073987,909.036716,908.479959,903.603100,902.340689,...,17628.024532,17571.499833,19937.936498,20058.881766,23751.448674,23842.097084,29703.261246,29895.408360,30080.316691,30012.988439
"(0.01, 0.9)",2505.372667,2508.087243,1697.776612,1696.511788,1064.763850,1065.255777,641.502053,640.844506,637.719307,637.424787,...,12027.502573,12188.385836,13806.375904,13957.108567,16236.157303,16199.876510,20626.824924,20578.564161,20415.088585,20496.342662
"(0.1, 0.9)",1311.493233,1310.688477,1004.838066,1004.446919,594.523596,594.870154,360.855188,360.524375,358.649208,358.521786,...,6291.972281,6270.298701,7021.269254,7185.586989,8439.055794,8415.833503,10533.267323,10506.235142,10484.199418,10592.190083


In [5]:
signal_pcles  = ['atm', 'hep', 'dsnb']

In [13]:
df_asimov =pd.DataFrame(index = [ 'z_thrd', 'power_z_alter', 'z_alter_avg'],
                         columns  =[signal_pcle + '_'+detector+'_'+metallicity+'_alpha'+str(alpha) for signal_pcle in signal_pcles 
                                 for detector in detectors 
                                  for  metallicity in metallicities
                                    for alpha in alphas
                                   ]
                        )
df_asimov

,atm_SURF_high_alpha0.001,atm_SURF_high_alpha0.01,atm_SURF_high_alpha0.1,atm_SURF_low_alpha0.001,atm_SURF_low_alpha0.01,atm_SURF_low_alpha0.1,atm_CJPL_high_alpha0.001,atm_CJPL_high_alpha0.01,atm_CJPL_high_alpha0.1,atm_CJPL_low_alpha0.001,...,dsnb_LNGS_high_alpha0.1,dsnb_LNGS_low_alpha0.001,dsnb_LNGS_low_alpha0.01,dsnb_LNGS_low_alpha0.1,dsnb_SNOlab_high_alpha0.001,dsnb_SNOlab_high_alpha0.01,dsnb_SNOlab_high_alpha0.1,dsnb_SNOlab_low_alpha0.001,dsnb_SNOlab_low_alpha0.01,dsnb_SNOlab_low_alpha0.1
z_thrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
power_z_alter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
z_alter_avg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:



binned_etaijk =  pd.read_csv('atmhepdsnb_Xe_binnedETAijk_optimized_consecutive_thrd0_S1S2binning.csv', index_col = 0)


for signal_pcle in signal_pcles:
    for detector in detectors:
        for metallicity in metallicities:
            null_ijkappa = np.array(binned_etaijk[signal_pcle + '_'+detector+'_'+metallicity + '_null_i'])
            alter_ijkappa = np.array(binned_etaijk[signal_pcle + '_'+detector+'_'+metallicity + '_alter_i'])

            null_ijkappa = null_ijkappa[~np.isnan(null_ijkappa)]
            alter_ijkappa = alter_ijkappa[~np.isnan(alter_ijkappa)]
            
            null_sum_ijkappa = sum(null_ijkappa[~np.isnan(null_ijkappa)])
            alter_sum_ijkappa = sum(alter_ijkappa[~np.isnan(alter_ijkappa)])
            for alpha in alphas:
                
                index_ab = str((alpha, power))

                DT = DT_pd.loc[index_ab][signal_pcle+'_'+detector+'_'+metallicity]

                qs_simnull, qs_simalter = get_pois_q(null_ijkappa, alter_ijkappa, DT, 
                                                     runtime = 10000, Data_null = True, Data_alter = True)
                #set negative to 0
                qs_simnull[qs_simnull<0] = 0
                qs_simalter[qs_simalter<0] = 0
                
                zs_simnull = np.sqrt(qs_simnull)
                zs_simalter = np.sqrt(qs_simalter)
                
                avg_zs_simalter = np.average(zs_simalter)
                
                z_alpha = np.percentile(zs_simnull, (1-alpha)*100)
                power_zalter = len(zs_simalter[zs_simalter>z_alpha])/len(zs_simalter)
                #check 
                E_N_null = null_sum_ijkappa * DT
                E_N_alter = alter_sum_ijkappa * DT
                sim_N_alter = np.random.poisson(E_N_alter, 10000)

                N_alpha_exp = poisson.ppf(round(1-alpha, 3), E_N_null)
                power_exp = len(sim_N_alter[sim_N_alter>N_alpha_exp])/len(sim_N_alter)

                df_asimov[signal_pcle + '_'+detector+'_'+metallicity+'_alpha'+str(alpha)]['z_thrd'] = z_alpha
                df_asimov[signal_pcle + '_'+detector+'_'+metallicity+'_alpha'+str(alpha)]['power_z_alter'] = power_zalter
                df_asimov[signal_pcle + '_'+detector+'_'+metallicity+'_alpha'+str(alpha)]['z_alter_avg'] = avg_zs_simalter
                
                print(signal_pcle, detector, metallicity, DT,
                      '\nalpha = ', alpha, 
                      'z_alpha = ', z_alpha, 
                      ',', sum(zs_simalter>0), 'positive zs_simalter, ', sum(zs_simnull>0), 'positive zs_simnull'
                      '\navg_zs_simalter = ', avg_zs_simalter, 
                      '\nfraction of zs_simnull > z_alpha :', round(len(zs_simnull[zs_simnull>z_alpha])/len(zs_simnull), 4), 

                      '\nfraction of zs_simalter > z_alpha :', power_zalter , 
                      'check sim_power = ', round(power_exp,  2), '\n\n')




atm SURF high 909.0367163145132 
alpha =  0.001 z_alpha =  2.6290000877869875 , 9824 positive zs_simalter,  136 positive zs_simnull
avg_zs_simalter =  4.682212179499299 
fraction of zs_simnull > z_alpha : 0.001 
fraction of zs_simalter > z_alpha : 0.9279 check sim_power =  0.9 


atm SURF high 641.5020532799832 
alpha =  0.01 z_alpha =  1.7584460966618627 , 9619 positive zs_simalter,  315 positive zs_simnull
avg_zs_simalter =  3.8721268053982145 
fraction of zs_simnull > z_alpha : 0.01 
fraction of zs_simalter > z_alpha : 0.9207 check sim_power =  0.9 


atm SURF high 360.8551875171093 
alpha =  0.1 z_alpha =  0.0 , 8991 positive zs_simalter,  709 positive zs_simnull
avg_zs_simalter =  2.8053171314027363 
fraction of zs_simnull > z_alpha : 0.0709 
fraction of zs_simalter > z_alpha : 0.8991 check sim_power =  0.9 


atm SURF low 908.4799588900308 
alpha =  0.001 z_alpha =  2.6974505024639694 , 9827 positive zs_simalter,  133 positive zs_simnull
avg_zs_simalter =  4.711594731587357 
frac

hep SURF high 6733.101851851853 
alpha =  0.001 z_alpha =  2.785989808378638 , 9954 positive zs_simalter,  60 positive zs_simnull
avg_zs_simalter =  5.011791717150052 
fraction of zs_simnull > z_alpha : 0.001 
fraction of zs_simalter > z_alpha : 0.9648 check sim_power =  0.9 


hep SURF high 4587.896750683268 
alpha =  0.01 z_alpha =  1.550734665170186 , 9810 positive zs_simalter,  190 positive zs_simnull
avg_zs_simalter =  4.089889329093778 
fraction of zs_simnull > z_alpha : 0.01 
fraction of zs_simalter > z_alpha : 0.9648 check sim_power =  0.9 


hep SURF high 2316.827027027028 
alpha =  0.1 z_alpha =  0.0 , 9341 positive zs_simalter,  626 positive zs_simnull
avg_zs_simalter =  2.8103591651845186 
fraction of zs_simnull > z_alpha : 0.0626 
fraction of zs_simalter > z_alpha : 0.9341 check sim_power =  0.9 


hep SURF low 5203.981244417982 
alpha =  0.001 z_alpha =  2.395514848764043 , 9949 positive zs_simalter,  46 positive zs_simnull
avg_zs_simalter =  5.026674126066784 
fraction o

dsnb SURF high 29703.26124638877 
alpha =  0.001 z_alpha =  2.9776803446557873 , 9844 positive zs_simalter,  155 positive zs_simnull
avg_zs_simalter =  4.292947878114741 
fraction of zs_simnull > z_alpha : 0.001 
fraction of zs_simalter > z_alpha : 0.8801 check sim_power =  0.9 


dsnb SURF high 20626.82492431887 
alpha =  0.01 z_alpha =  1.8738599362517372 , 9647 positive zs_simalter,  321 positive zs_simnull
avg_zs_simalter =  3.536925904661086 
fraction of zs_simnull > z_alpha : 0.01 
fraction of zs_simalter > z_alpha : 0.9084 check sim_power =  0.9 


dsnb SURF high 10533.267323250087 
alpha =  0.1 z_alpha =  0.0 , 9027 positive zs_simalter,  989 positive zs_simnull
avg_zs_simalter =  2.4423064958551457 
fraction of zs_simnull > z_alpha : 0.0989 
fraction of zs_simalter > z_alpha : 0.9027 check sim_power =  0.9 


dsnb SURF low 29895.40836012862 
alpha =  0.001 z_alpha =  2.824297870162749 , 9858 positive zs_simalter,  128 positive zs_simnull
avg_zs_simalter =  4.313695492534292 
f

In [15]:
file_name = 'atmhepdsnb_Xe_cfASIMOV_optimized_consecutive_thrd0_S1S2binning.csv'
file_name


'atmhepdsnb_Xe_cfASIMOV_optimized_consecutive_thrd0_S1S2binning.csv'

In [16]:
df_asimov

,atm_SURF_high_alpha0.001,atm_SURF_high_alpha0.01,atm_SURF_high_alpha0.1,atm_SURF_low_alpha0.001,atm_SURF_low_alpha0.01,atm_SURF_low_alpha0.1,atm_CJPL_high_alpha0.001,atm_CJPL_high_alpha0.01,atm_CJPL_high_alpha0.1,atm_CJPL_low_alpha0.001,...,dsnb_LNGS_high_alpha0.1,dsnb_LNGS_low_alpha0.001,dsnb_LNGS_low_alpha0.01,dsnb_LNGS_low_alpha0.1,dsnb_SNOlab_high_alpha0.001,dsnb_SNOlab_high_alpha0.01,dsnb_SNOlab_high_alpha0.1,dsnb_SNOlab_low_alpha0.001,dsnb_SNOlab_low_alpha0.01,dsnb_SNOlab_low_alpha0.1
z_thrd,2.629,1.758446,0.0,2.697451,1.650226,0.0,2.722446,1.734862,0.0,2.689782,...,0.0,2.647851,1.933093,0.0,2.917362,1.923388,0.0,2.792211,1.934688,0.101252
power_z_alter,0.9279,0.9207,0.8991,0.9261,0.9256,0.9003,0.9208,0.9254,0.904,0.9238,...,0.9022,0.9162,0.9063,0.8999,0.8914,0.9035,0.8953,0.9021,0.8979,0.8964
z_alter_avg,4.682212,3.872127,2.805317,4.711595,3.875651,2.811417,4.632844,3.848589,2.692546,4.634662,...,2.445047,4.307918,3.541358,2.450714,4.333199,3.516826,2.413839,4.314688,3.500823,2.407731


In [17]:
df_asimov.to_csv(file_name)